In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import os
import numpy as np
import pandas as pd
from google.colab import drive
from transformers import GPT2Tokenizer, GPT2LMHeadModel

drive.mount('/content/drive')
project_path = './drive/MyDrive/Colab Notebooks/GPT_community/'

def generate_multiple_responses(model, tokenizer, prompt, num_return_sequences=2):
  """For a given prompt, output multiple different responses from a model.
  
  Args:
    model (GPT2LMHeadModel): Model to generate responses from
    tokenizer (GPT2Tokenizer): Tokenizer for model
    prompt (str): Prompt for model to respond to
    num_return_sequences (int): Number of reponses to return

  Returns:
    list[str]: Multiple different responses to prompt
  """

  input_ids = tokenizer(prompt, return_tensors="pt").input_ids

  gen_ids_list = model.generate(
    input_ids,
    num_return_sequences = num_return_sequences,
    do_sample=True,
    temperature=0.9,
    max_length=100,
    pad_token_id=tokenizer.eos_token_id
  )

  gen_text_list = tokenizer.batch_decode(gen_ids_list)

  return gen_text_list

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
rewards_file = os.path.join(project_path, 'data/rewards.csv')
df_rewards = pd.read_csv(rewards_file)

prompts_file = os.path.join(project_path, 'data/brighton_philosophy_prompts.txt')
with open(prompts_file) as file:
    prompts = [line.rstrip() for line in file]

In [6]:
selected_prompt = np.random.choice(prompts)
print(f"Question: {selected_prompt}\n")

gen_text_list = generate_multiple_responses(model, tokenizer, selected_prompt, num_return_sequences=3)

for n, response in enumerate(gen_text_list):
  print(f"Response {n+1}: {response}\n")

Question: When should you change your mind?

Response 1: When should you change your mind? You might find it hard to put it all into perspective until you're trying to figure out what are the rules you should follow.

When do you start to think differently about your parenting?

How about when you get a new roommate? You might find that it's easier to be open and honest in your assessment about your son's behavior.

You might want to get the results of a small test that shows how often you can use your parental

Response 2: When should you change your mind?

You don't have to change your mind. I am happy that the majority of the people in the world are not happy with what we are doing. If you don't believe me, your child may start to doubt you. But if you are doing something about these issues, you may just end up changing your mind.

People who care deeply about this issue are most often kids. They don't understand why people feel alienated. Sometimes, they just

Response 3: When shou

In [7]:
"""Rank the three responses and define worst as reward=0, middle as reward=0.5, and best as reward=1"""

best_response_number = 2
middle_response_number = 1
worst_response_number = 3

df_responses = pd.DataFrame({'prompt':[selected_prompt, selected_prompt, selected_prompt],
                             'response':[gen_text_list[best_response_number-1], gen_text_list[middle_response_number-1], gen_text_list[worst_response_number-1]],
                             'reward':[1, 0.5, 0]})
df_rewards = pd.concat([df_rewards, df_responses])
df_rewards.tail()

,prompt,response,reward
1,"Does life have meaning? if yes, what is it? an...","Does life have meaning? if yes, what is it? an...",0.5
2,"Does life have meaning? if yes, what is it? an...","Does life have meaning? if yes, what is it? an...",0.0
0,When should you change your mind?,When should you change your mind?\n\nYou don't...,1.0
1,When should you change your mind?,When should you change your mind? You might fi...,0.5
2,When should you change your mind?,When should you change your mind?\n\nIf you ne...,0.0


In [8]:
# Note: want to change to saving a Dataset object at some point?
df_rewards.to_csv(rewards_file, index=False)